# NOTES
TODO: <br>
- figure out how to distinguish animals
- finish FINAL event based on empty spaces detection (if none => show text "game won" in the top left corner of frame)
- try to match woolf on dices using Canny? 1. check how boarding box of dice looks like, how the woolf looks like to see if it makes sense. if it does, take rotation into concideration . Try matchTemplate or ORB? maybe something else 
- detect TRADE box. has to be 1. outside of board (not yet detectable) 2. maybe check for black rectangle? [dont use any corner-like, as it wont work on dificult. Can only get in correspondence to board. maybe as a {big} square outside of board] in the trade box search if theres a circle (as defined in function that detects circles [circle size has to be defined in correspondence to the board]. If there is a circle=> trigger "TRADING" event and write it on top left corner of the frame)
- try to merge tracking dice with final detection. Change the code so that its all in functions, triggered in each frame and readable.
#### do not think that matching sheep to sheep or figuring out anything on dice is possible. stick to shapes and colours. 



### What has to be done: 
Events: <br>
1. Trading
2. Wolf 
3. Rolling dice / getting animal
4. Dog buying
5. Final

<br>
Possible events:

- Fox (hard to detect)

Tracking: <br>
1. Dice
2. Board??? computationally expensive, maybe will make sense for difficult. Not necessary. Do not try unless all else is finished



#### project description:

- perp of dataset
- persentation of results
- detection of Dice and animals, board (position can be calculated from board position)
- tracking movement of dice

<Br> 
For 4: 

- detection of 2 events ( corresponding message on the screeen)
<br><br>

For 5: 

- game status (# of animals)
- detection of 5 events
- track gameplay score (no score for us- singleplayer)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import os
import sys
import glob
import random

from pprint import pprint
from ipywidgets import Video

from PIL import Image
from PIL.ExifTags import TAGS

# opening video, basic characteristics

In [ ]:
game = cv2.VideoCapture("./materials/short.mp4")
if game.isOpened():
    print("video loaded")
    
game_width = int(game.get(3))
game_height = int(game.get(4))

print(game_width, game_height)

game_fps = game.get(cv2.CAP_PROP_FPS)
print(game_fps)

# opening image

In [ ]:
temp_o_1 = cv2.imread("./materials/cow2.png", cv2.IMREAD_GRAYSCALE)
temp_o_1_width, temp_o_1_height = temp_o_1.shape[:2]


# Dice detection

In [ ]:
import cv2
import numpy as np

video_path = "./materials/uppig.mov"
game = cv2.VideoCapture(video_path)
if game.isOpened():
    print("Video loaded")

target_width, target_height = 1024, 768

original_width = int(game.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(game.get(cv2.CAP_PROP_FRAME_HEIGHT))
game_fps = game.get(cv2.CAP_PROP_FPS)

# Scaling factors for resizing
scale_x = target_width / original_width
scale_y = target_height / original_height

#My colors
orange_lower = np.array([4, 66, 114])  # Lower bound for orange                 #change this if colors for dice dont work.
orange_upper = np.array([15, 252, 253])  # Upper bound for orange
blue_lower = np.array([90, 80, 100])    # Lower bound for light blue
blue_upper = np.array([120, 255, 255])   # Upper bound for light blue


output_path = "./output/track_1.avi"
output_video = cv2.VideoWriter(
    output_path,
    cv2.VideoWriter_fourcc(*"DIVX"),
    game_fps,
    (target_width, target_height),
)

trackers = []

def create_tracker(tracker_type="CSRT"):
    if tracker_type == "CSRT":
        return cv2.TrackerCSRT_create()
    elif tracker_type == "KCF":
        return cv2.TrackerKCF_create()
    return None

def draw_bbox_with_area(frame, bbox, color=(255, 255, 255)):
    """Draws a bounding box and displays its area."""
    p1 = (int(bbox[0]), int(bbox[1]))
    p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv2.rectangle(frame, p1, p2, color, 2, 1)
    text_position = (p1[0], max(0, p1[1] - 10))
    cv2.putText(
        frame,
        f"Dice",
        text_position,
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        color,
        1,
        cv2.LINE_AA,
    )

def is_square(bbox, tolerance=0.2):
    x, y, w, h = bbox
    aspect_ratio = w / h if w > h else h / w
    return 1 - tolerance <= aspect_ratio <= 1 + tolerance

def is_overlap(bbox1, bbox2):
    x1, y1, w1, h1 = bbox1
    x2, y2, w2, h2 = bbox2
    return not (x1 + w1 < x2 or x2 + w2 < x1 or y1 + h1 < y2 or y2 + h2 < y1)

def is_valid_color(frame, bbox, orange_lower, orange_upper, blue_lower, blue_upper, threshold=0.3):
    x, y, w, h = bbox
    roi = frame[y:y+h, x:x+w]  # Extract ROI
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)  # Convert ROI to HSV
    orange_mask = cv2.inRange(hsv_roi, orange_lower, orange_upper)
    blue_mask = cv2.inRange(hsv_roi, blue_lower, blue_upper)
    orange_pixels = cv2.countNonZero(orange_mask)
    blue_pixels = cv2.countNonZero(blue_mask)
    total_pixels = w * h
    return (orange_pixels + blue_pixels) / total_pixels >= threshold

# Start tracking
while game.isOpened():
    ret, frame = game.read()
    if not ret:
        break

    # Resize the frame
    frame = cv2.resize(frame, (target_width, target_height))
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Create masks for orange and blue dice
    orange_mask = cv2.inRange(hsv_frame, orange_lower, orange_upper)
    blue_mask = cv2.inRange(hsv_frame, blue_lower, blue_upper)
    combined_mask = cv2.bitwise_or(orange_mask, blue_mask)

    # Find contours
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detected_bboxes = []

    for contour in contours:
        area = cv2.contourArea(contour)
        if 1000 < area < 4500:  # Area thresholds
            x, y, w, h = cv2.boundingRect(contour)
            if is_square((x, y, w, h)):
                detected_bboxes.append((x, y, w, h, area))

    # Check for new dice to track
    for bbox_data in detected_bboxes:
        bbox = bbox_data[:4]
        area = bbox_data[4]
        is_new = True
        for tracker, tracked_bbox in trackers:
            if is_overlap(bbox, tracked_bbox):
                is_new = False
                break

        if is_new and is_valid_color(frame, bbox, orange_lower, orange_upper, blue_lower, blue_upper):
            tracker = create_tracker("KCF")
            tracker.init(frame, bbox)
            trackers.append((tracker, bbox))
            print(f"New dice detected and tracking initialized at {bbox}")

    # Update trackers
    for i, (tracker, _) in enumerate(trackers):
        ok, bbox = tracker.update(frame)
        if ok:
            if is_valid_color(frame, bbox, orange_lower, orange_upper, blue_lower, blue_upper):
                trackers[i] = (tracker, bbox)
                area = bbox[2] * bbox[3]
                draw_bbox_with_area(frame, bbox, (0, 255, 0))
            else:
                print(f"Tracker {i} lost valid target at {bbox}")
                trackers[i] = None
        else:
            print(f"Tracker {i} failed at {bbox}")
            trackers[i] = None

    # Remove invalid trackers
    trackers = [t for t in trackers if t is not None]

    # Save frame to output video
    output_video.write(frame)
    cv2.imshow("Dice Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

game.release()
output_video.release()
cv2.destroyAllWindows()


Video loaded
New dice detected and tracking initialized at (934, 675, 90, 75)
Tracker 0 failed at (0, 0, 0, 0)
New dice detected and tracking initialized at (974, 655, 50, 54)
Tracker 0 failed at (0, 0, 0, 0)
New dice detected and tracking initialized at (846, 601, 60, 56)
New dice detected and tracking initialized at (759, 580, 68, 62)
Tracker 0 failed at (0, 0, 0, 0)
Tracker 0 failed at (0, 0, 0, 0)
New dice detected and tracking initialized at (730, 555, 67, 61)
New dice detected and tracking initialized at (828, 553, 54, 61)
Tracker 0 failed at (0, 0, 0, 0)
Tracker 1 failed at (0, 0, 0, 0)
New dice detected and tracking initialized at (829, 530, 62, 66)
New dice detected and tracking initialized at (699, 514, 64, 59)
Tracker 1 failed at (0, 0, 0, 0)
New dice detected and tracking initialized at (667, 471, 66, 59)
Tracker 0 failed at (0, 0, 0, 0)
New dice detected and tracking initialized at (838, 494, 74, 67)
Tracker 0 failed at (0, 0, 0, 0)
New dice detected and tracking initializ

# detecting empty animal spaces (USE FOR FINALE DETECTION)

In [11]:
import cv2
import numpy as np
from collections import Counter

def get_dominant_color(image, mask):
    # Extract colors within the mask
    colors = image[mask == 255]
    if len(colors) == 0:
        return (0, 0, 0)  # Default if no pixels found in mask

    # Convert to a list of tuples and count frequencies
    colors = [tuple(color) for color in colors]
    most_common_color = Counter(colors).most_common(1)[0][0]

    return most_common_color

def detect_circles_and_colors(video_path):
    # Open video file
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)

        # Detect circles using HoughCircles
        circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30, param1=200, param2=100, minRadius=10, maxRadius=100)

        if circles is not None:
            circles = np.uint16(np.around(circles))
            for circle in circles[0, :]:
                x, y, r = circle

                # Create a circular mask for the detected circle
                mask = np.zeros_like(gray, dtype=np.uint8)
                cv2.circle(mask, (x, y), r, 255, -1)

                # Get dominant color inside the circle
                dominant_color = get_dominant_color(frame, mask)
                color_name = f"RGB({dominant_color[2]}, {dominant_color[1]}, {dominant_color[0]})"

                # Draw the circle and bounding box
                cv2.circle(frame, (x, y), r, (0, 255, 0), 2)
                cv2.rectangle(frame, (x - r, y - r - 20), (x + r, y - r), (0, 255, 0), -1)

                # Put text (dominant color)
                cv2.putText(frame, color_name, (x - r, y - r - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

        # Show the frame
        cv2.imshow("Circles and Colors", frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Replace 'video.mp4' with the path to your video file
detect_circles_and_colors('./materials/upright.mp4')


# Detecting all circles (animals) 

In [ ]:
import cv2 
import numpy as np

def detect_board_and_calibrate(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        # Approximate the contour to a polygon
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        if len(approx) == 4:
            board = approx
            return board

    return None

def get_dominant_color(frame, mask):
    # Convert the frame to the LAB color space
    lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)

    # Apply the mask to the LAB image
    mask_flat = mask.flatten()
    lab_flat = lab.reshape((-1, 3))
    color_mask = lab_flat[mask_flat == 255]

    # Calculate the mean of the masked region
    mean_color = np.mean(color_mask, axis=0)
    return mean_color

# from scipy.stats import mode

# def get_dominant_color(frame, mask):
#     # Convert the frame to the LAB color space
#     lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)

#     # Apply the mask to the LAB image
#     mask_flat = mask.flatten()
#     lab_flat = lab.reshape((-1, 3))
#     color_mask = lab_flat[mask_flat == 255]

#     # Find the mode (most frequent color) in each channel
#     dominant_color = mode(color_mask, axis=0, keepdims=False).mode
#     return dominant_color



def compute_circle_size_from_board(board):
    # Compute board dimensions (assumes rectangle in perspective)
    distances = [
        np.linalg.norm(board[i][0] - board[(i+1) % 4][0]) for i in range(4)
    ]
    avg_distance = sum(distances) / len(distances)

    # Define circle size based on average board dimension
    return avg_distance / 12  

def detect_circles_in_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Read the first frame to calibrate
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read the first frame.")
        cap.release()
        return
    
    # Detect the board
    board = detect_board_and_calibrate(frame)
    if board is None:
        print("Error: Could not detect the board.")
        cap.release()
        return

    # Compute circle size based on the board dimentions
    circle_size = compute_circle_size_from_board(board)
    tolerance = 0.07 * circle_size  

    while True:
        # Read next frame
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)

        # Detect circles using HoughCircles
        circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50, param1=70, param2=30, minRadius=int(circle_size - tolerance), maxRadius=int(circle_size + tolerance))


        if circles is not None:
            circles = np.uint16(np.around(circles))
            
            detected_circles = []
            for circle in circles[0, :]:
                x, y, r = circle

                # Create a circular mask for the detected circle
                mask = np.zeros_like(gray, dtype=np.uint8)
                cv2.circle(mask, (x, y), r, 255, -1)

                # Get dominant color inside the circle
                dominant_color = get_dominant_color(frame, mask)
                print(dominant_color)
                color_name = f"({dominant_color[2]}, {dominant_color[1]}, {dominant_color[0]})"

                print(dominant_color)
                # Draw the circle and bounding box
                cv2.circle(frame, (x, y), r, (dominant_color[2], dominant_color[1], dominant_color[0]), 4)
                # cv2.rectangle(frame, (x - r, y - r - 20), (x + r, y - r), (0, 255, 0), -1)

                # Put text (dominant color)
                # cv2.putText(frame, color_name, (x - r, y - r - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                
#THIS CODE CHECKS FOR OVERLAPPING, BUT DOESNT GET THE COLOR OF THE CIRCLE 
        # if circles is not None:
        #     circles = np.uint16(np.around(circles))
        #     detected_circles = []

        #     for circle in circles[0, :]:
        #         x, y, r = circle

        #         # Check for overlapping circles
        #         overlap = False
        #         for detected in detected_circles:
        #             dx, dy, dr = detected
        #             if np.sqrt((x - dx) ** 2 + (y - dy) ** 2) < 50:
        #                 overlap = True
        #                 break

        #         if not overlap:
        #             detected_circles.append((x, y, r))

        #             # Draw the circle and bounding box
        #             cv2.circle(frame, (x, y), r, (0, 255, 0), 2)
        #             cv2.rectangle(frame, (x - r, y - r - 20), (x + r, y - r), (0, 255, 0), -1)

        #             # Put text (circle size)
        #             size_text = f"Size: {r}"
        #             cv2.putText(frame, size_text, (x - r, y - r - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

        cv2.imshow("Circles in Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


detect_circles_in_video('./materials/upright.mp4')


[188.15673636 135.8247131  143.19933973]
[188.15673636 135.8247131  143.19933973]
[205.1601119  122.11601119 123.82211618]
[205.1601119  122.11601119 123.82211618]
[187.61214415 134.58828369 144.0287971 ]
[187.61214415 134.58828369 144.0287971 ]
[189.95803851 135.27678131 143.73358565]
[189.95803851 135.27678131 143.73358565]
[187.87757117 134.74477538 144.47951292]
[187.87757117 134.74477538 144.47951292]
[137.60135819 111.85897992 150.9810721 ]
[137.60135819 111.85897992 150.9810721 ]
[141.91864151 110.9314717  148.65086792]
[141.91864151 110.9314717  148.65086792]
[198.49070265 134.49366464 142.00822774]
[198.49070265 134.49366464 142.00822774]
[193.14195881 122.96840119 126.3188178 ]
[193.14195881 122.96840119 126.3188178 ]
[135.57177358 144.94943396 153.01449057]
[135.57177358 144.94943396 153.01449057]
[145.12463607 122.98738389 132.90711216]
[145.12463607 122.98738389 132.90711216]
[188.08491032 134.566727   144.07536613]
[188.08491032 134.566727   144.07536613]
[135.01831059 11

# Detecting board in each frame, circles main color (for all circles) ( a lot of false positives )

In [ ]:
### THIS DETECTS BOARD IN EACH FRAME, DETECT CIRCLES AND THEIR COLORS
import cv2 
import numpy as np
from scipy.stats import mode

def detect_board(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9, 9), 3)
    edges = cv2.Canny(gray, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.imshow("sd", edges)

    for contour in contours:
        # Approximate the contour to a polygon
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        if len(approx) == 4:
            # Calculate the bounding box
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)

            # Check aspect ratio and solidity
            if 0.95 <= aspect_ratio <= 1.05:  # Allowing some tolerance for a square
                if cv2.isContourConvex(approx):  # Check contour solidity
                    board = approx
                    return board

    return None



def get_dominant_color(frame, mask):
    # Flatten the mask and the frame for easier indexing
    mask_flat = mask.flatten()
    frame_flat = frame.reshape((-1, 3))

    # Select the pixels in the masked region
    color_mask = frame_flat[mask_flat == 255]

    # Check if any pixels are selected
    if color_mask.size == 0:
        return (0, 0, 0)  # Default to black if the mask region is empty

    # Find the mode (most frequent color) in each channel
    dominant_color = mode(color_mask, axis=0, keepdims=False).mode

    # Ensure the result is a tuple
    return tuple(map(int, dominant_color))


def compute_circle_size_from_board(board):
    # Compute board dimensions (assumes rectangle in perspective)
    distances = [
        np.linalg.norm(board[i][0] - board[(i+1) % 4][0]) for i in range(4)
    ]
    avg_distance = sum(distances) / len(distances)

    # Define circle size based on average board dimension
    return avg_distance / 12  

def detect_circles_in_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Read the first frame to calibrate
    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read the first frame.")
        cap.release()
        return
    
    # Detect the board
    board = detect_board(frame)
    print(board)
    if board is None:
        print("Error: Could not detect the board.")
        cap.release()
        return

    # Compute circle size based on the board dimentions
    circle_size = compute_circle_size_from_board(board)
    tolerance = 0.07 * circle_size  

    while True:
        # Read next frame
        ret, frame = cap.read()
        if not ret:
            break
        board = detect_board(frame)
        if board is not None:
            cv2.polylines(frame, [board], isClosed=True, color=(255, 0, 255), thickness=4)
        

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur
        blurred = cv2.GaussianBlur(gray, (9, 9), 2)

        # Detect circles using HoughCircles
        circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=50, param1=70, param2=30, minRadius=int(circle_size - tolerance), maxRadius=int(circle_size + tolerance))

        if circles is not None:
            circles = np.uint16(np.around(circles))
            
            detected_circles = []
            for circle in circles[0, :]:
                x, y, r = circle

                # Create a circular mask for the detected circle
                mask = np.zeros_like(gray, dtype=np.uint8)
                cv2.circle(mask, (x, y), r, 255, -1)

                # Get dominant color inside the circle
                dominant_color = get_dominant_color(frame, mask)
                # color_name = f"({dominant_color[2]}, {dominant_color[1]}, {dominant_color[0]})"
                color_name = f"({dominant_color})"
                
                # Draw the circle and bounding box
                cv2.circle(frame, (x, y), r, (int(dominant_color[0]), int(dominant_color[1]), int(dominant_color[2])), 4)

            
        cv2.imshow("Circles in Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# detect_circles_in_video('./materials/upright.mp4')
detect_circles_in_video('./materials/uppig.mov')



# Detecting dogs (only circles around them done)

In [7]:
import cv2
import numpy as np

def detect_board(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (9, 9), 2)
    edges = cv2.Canny(gray, 50, 150)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        if len(approx) == 4:
            x, y, w, h = cv2.boundingRect(approx)
            aspect_ratio = w / float(h)

            if 0.95 <= aspect_ratio <= 1.05 and cv2.isContourConvex(approx):
                return approx, x, y, w, h  # Returning bounding box dimensions

    return None, None, None, None, None

def detect_circles_in_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read the first frame.")
        cap.release()
        return

    board, x, y, w, h = detect_board(frame)
    if board is None:
        print("Error: Could not detect the board.")
        cap.release()
        return

    board_corners = board.reshape(4, 2)
    bottom_left = board_corners[2]  # Bottom-left corner
    bottom_right = board_corners[3]  # Bottom-right corner
    square_side = int(w * 0.23)  # 1/5 of the board width/height
    roi_size = (square_side, square_side)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        roi_bottom_right = (
            int(bottom_left[0] + (square_side//7)),
            int(bottom_left[1] - (square_side//7) - roi_size[1]),
            roi_size[0],
            roi_size[1]
        )
        roi_bottom_left = (
            int(bottom_right[0] - (square_side//7)- roi_size[0]),
            int(bottom_right[1] - (square_side//7) - roi_size[1]),
            roi_size[0],
            roi_size[1]
        )

        # Extract ROIs
        roi_frame_left = frame[roi_bottom_right[1]:roi_bottom_right[1] + roi_bottom_right[3], 
                               roi_bottom_right[0]:roi_bottom_right[0] + roi_bottom_right[2]]
        roi_frame_right = frame[roi_bottom_left[1]:roi_bottom_left[1] + roi_bottom_left[3], 
                                roi_bottom_left[0]:roi_bottom_left[0] + roi_bottom_left[2]]

        # Convert to grayscale for Hough Circles
        roi_gray_left = cv2.cvtColor(roi_frame_left, cv2.COLOR_BGR2GRAY)
        roi_gray_right = cv2.cvtColor(roi_frame_right, cv2.COLOR_BGR2GRAY)

        circles_left = cv2.HoughCircles(roi_gray_left, cv2.HOUGH_GRADIENT, 1, roi_size[0] / 4,
                                         param1=50, param2=30, minRadius=5, maxRadius=50)

        circles_right = cv2.HoughCircles(roi_gray_right, cv2.HOUGH_GRADIENT, 1, roi_size[0] / 4,
                                         param1=50, param2=30, minRadius=5, maxRadius=50)

#Drawing left dog
        if circles_left is not None:
            circles_left = np.uint16(np.around(circles_left))
            circles_left = sorted(circles_left[0, :], key=lambda c: -c[2])  # Sort by radius, descending to draw only the biggest circle
            if len(circles_left) > 0:
                largest_circle = circles_left[0]
                cv2.circle(roi_frame_left, (largest_circle[0], largest_circle[1]), largest_circle[2], (60, 103, 155), 2)
                cv2.circle(roi_frame_left, (largest_circle[0], largest_circle[1]), 2, (60, 103, 155), 3)

#Drawing right dog
        if circles_right is not None:
            circles_right = np.uint16(np.around(circles_right))
            circles_right = sorted(circles_right[0, :], key=lambda c: -c[2])  
            if len(circles_right) > 0:
                largest_circle = circles_right[0]
                cv2.circle(roi_frame_right, (largest_circle[0], largest_circle[1]), largest_circle[2], (60, 103, 155), 2)
                cv2.circle(roi_frame_right, (largest_circle[0], largest_circle[1]), 2, (60, 103, 155), 3)

        # Place the processed ROIs back into the frame
        frame[roi_bottom_right[1]:roi_bottom_right[1] + roi_bottom_right[3], 
              roi_bottom_right[0]:roi_bottom_right[0] + roi_bottom_right[2]] = roi_frame_left
        frame[roi_bottom_left[1]:roi_bottom_left[1] + roi_bottom_left[3], 
              roi_bottom_left[0]:roi_bottom_left[0] + roi_bottom_left[2]] = roi_frame_right

#Visualisation of boxes of search for dogs
        cv2.rectangle(frame, roi_bottom_right[:2], (roi_bottom_right[0] + roi_size[0], roi_bottom_right[1] + roi_size[1]), (0, 255, 0), 2)
        cv2.rectangle(frame, roi_bottom_left[:2], (roi_bottom_left[0] + roi_size[0], roi_bottom_left[1] + roi_size[1]), (0, 255, 0), 2)

        cv2.imshow("Circles in Video", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage:
detect_circles_in_video('./materials/upright.mp4')   #has 2 dogs at the start
# detect_circles_in_video('./materials/uppig.mov')    #NO dogs


C:\Users\mary9\AppData\Local\Temp\ipykernel_23512\3671512606.py:85: RuntimeWarning: overflow encountered in scalar negative
  circles_left = sorted(circles_left[0, :], key=lambda c: -c[2])  # Sort by radius, descending to draw only the biggest circle
C:\Users\mary9\AppData\Local\Temp\ipykernel_23512\3671512606.py:94: RuntimeWarning: overflow encountered in scalar negative
  circles_right = sorted(circles_right[0, :], key=lambda c: -c[2])
